In [0]:
%pip install "dlt[databricks]"

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.7 MB/s eta 0:00:00
  Created wheel for thrift: filename=thrift-0.20.0-cp312-cp312-linux_x86_64.whl size=412853 sha256=5681b97410937c2ce46ebfc790ea98a1ea8a52c79b59db506853bba66b49d04c
  Stored in directory: /home/spark-cf3ba049-00de-4b08-bdb8-7b/.cache/pip/wheels/57/4d/9e/539ef8e052f8e2db244ddf8ac1f2e623b4663f2ee233ffafb1
Successfully built thrift
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
import sys
import types

# 1 Drop Databricks' post-import hook
sys.meta_path = [h for h in sys.meta_path if 'PostImportHook' not in repr(h)]

# 2 Purge half-initialized Delta-Live-Tables modules
for name, module in list(sys.modules.items()):
    if not isinstance(module, types.ModuleType):
        continue
    if (getattr(module, '__file__', '') or '').startswith('/databricks/spark/python/dlt'): del sys.modules[name]

In [0]:
import dlt
from pyspark.sql import functions as F
from pyspark.sql.types import *

BASE_URL = "https://fakeapi.net" 

def fetch(endpoint: str):
    r = requests.get(f"{BASE_URL}{endpoint}", timeout=30)
    r.raise_for_status()
    return r.json()

@dlt.table(
  name="products",
  comment="Raw products pulled from Fake E-commerce API",
)
@dlt.expect_all_or_drop({"id_not_null": "id IS NOT NULL"})
def bronze_products():
    data = fetch("/api/v1/ecommerce/products")
    return spark.createDataFrame(data)

@dlt.table(
  name="users",
  comment="Raw users"
)
def bronze_users():
    data = fetch("/api/v1/ecommerce/users")
    return spark.createDataFrame(data)

@dlt.table(
  name="orders",
  comment="Raw orders"
)
def bronze_orders():
    data = fetch("/api/v1/ecommerce/orders")
    return spark.createDataFrame(data)

Unexpected internal error when monkey patching dlt module: cannot import name 'overrides' from partially initialized module 'dlt' (most likely due to a circular import) (/local_disk0/.ephemeral_nfs/envs/pythonEnv-22441f72-858b-4fd4-8cc1-f26574e0c12a/lib/python3.12/site-packages/dlt/__init__.py)


---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
File <command-6793294230386396>, line 12
      9     r.raise_for_status()
     10     return r.json()
---> 12 @dlt.table(
     13   name="products",
     14   comment="Raw products pulled from Fake E-commerce API",
     15 )
     16 @dlt.expect_all_or_drop({"id_not_null": "id IS NOT NULL"})
     17 def bronze_products():
     18     data = fetch("/api/v1/ecommerce/products")
     19     return spark.createDataFrame(data)

AttributeError: module 'dlt' has no attribute 'table'